<a href="https://colab.research.google.com/github/KonstantinaT/Predictor/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SOMATIC MUTATIONS

### Libraries and functions

In [ ]:
import pandas as pd
!pip install Biopython
import Bio
import os
import shutil
import copy
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import gc
!pip install Biopython
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import requests
pd.set_option('display.max_rows', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 14.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def notNaN(num):
    return num == num

In [ ]:
def load_uniprot_fasta(identifier): #loads fasta file for a given UniProt identifier
    #if '-' in identifier:
    #  ident = identifier[:-2]
    #  which_isoform = int(identifier[-1]) - 1
    #  link = "http://www.uniprot.org/uniprot/" + ident + ".fasta?include=yes"
    #else:
    link = "http://www.uniprot.org/uniprot/" + identifier + ".fasta"
    
    str_data = requests.get(link).content.decode('utf-8')
    fasta = str_data.split('>')
    fasta_all=[]
    for seq in fasta[1:]:
      temp = seq.splitlines()[1:]
      temp = ''.join(temp)
      fasta_all.append(temp)
    return fasta_all[0]

In [ ]:
def AAconvert(AminoAcid):

  aa = {
      'Ala' : 'A',
      'Arg' : 'R',
      'Asn' : 'N',
      'Asp' : 'D',
      'Cys' : 'C',
      'Glu' : 'E',
      'Gln' : 'Q',
      'Gly' : 'G',
      'His' : 'H',
      'Ile' : 'I',
      'Leu' : 'L',
      'Lys' : 'K',
      'Met' : 'M',
      'Phe' : 'F',
      'Pro' : 'P',
      'Ser' : 'S',
      'Thr' : 'T',
      'Trp' : 'W',
      'Tyr' : 'Y',
      'Val' : 'V',
      'Ter' : '*'

  }
  return aa[AminoAcid]

In [ ]:
def load_uniprot_fasta_all(identifier): #loads fasta file for a given UniProt identifier
    link = "https://rest.uniprot.org/uniprotkb/search?format=fasta&includeIsoform=true&query=accession%3A" + identifier  # canonical & isoforms
    str_data = requests.get(link).content.decode('utf-8')

    fasta = str_data.split('>')
    fasta_all=[]
    isoforms =[]
    for seq in fasta[1:]:
      isoforms.append(seq.splitlines()[0].split('|')[1])
      temp = seq.splitlines()[1:]
      temp = ''.join(temp)
      fasta_all.append(temp)
    return fasta_all, isoforms

In [ ]:
#Change the AA of the gene sequence 
def changeAAnew(gene_name, original, target, position, sequence):
    #position = int(position)
    position -= 1 # because python indexation
    sequence = list(sequence)
    #print(len(sequence))
    #print(position)
    if sequence[position] == original:
        sequence[position]= target
        transcript = ''.join(sequence)
        return transcript
    else:
        tru = sequence[position]
        print('in gene {} the original {}, in position {} not equal to: {}'.format(gene_name, original, position+1, tru) )

## Missense, nonsense and silent mutations from COSMIC cmc_export, without uniprot / trembl ID

### Cmc_mutations import

In [ ]:
# Read transcript consensus (glioma_All_cosmic.ipynb)
with open("/content/drive/MyDrive/my_colab/*2ndYear/transcript_sequences.csv", 'rb') as fp:
    gene_sequence = pickle.load(fp)

In [ ]:
cmc_all = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/cmc_export.csv",  delimiter = '\t') # COSMIC: cmc Tier 1,2,3, other mutations

# Filter missense / silent / nonsense mutations
missense_sil_nonsense_mut = cmc_all[(cmc_all['Mutation Description AA'] == 'Substitution - Missense') \
                              | (cmc_all['Mutation Description AA']== 'Substitution - coding silent') \
                                       | (cmc_all['Mutation Description AA']== 'Substitution - Nonsense')]

# creation of Tier1-2-3 mutations
Tier1_2_3 = missense_sil_nonsense_mut[(missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER'] == '1') \
                              | (missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER']== '2') \
                                       | (missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER']== '3')]
                             

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,24,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(Tier1_2_3_nodelins)

3530

In [ ]:
# Keep mutations in position <= 2000
Tier1_2_3 = Tier1_2_3.reset_index(drop=True)
count_more_1 =0
position, AA_orig, AA_targ, sequence, indices_to_excl = [], [], [], [], []
for i, mut in Tier1_2_3.iterrows():

    string = mut['Mutation AA']
    if 'delins' in string:
      indices_to_excl.append(i)
      continue
    pos_temp = int(string[3:-1])
    if pos_temp > 2000:
      indices_to_excl.append(i)
      continue
    else:
      wt_sequence = str(gene_sequence[mut['ACCESSION_NUMBER']])
      if wt_sequence.count('*') > 1: # if there is a codon mid-protein
          count_more_1 +=1
      mut_seq = changeAAnew(string[2], string[-1], pos_temp,wt_sequence)
  
      if mut_seq is not None:
        if '*' in mut_seq:
          temp = mut_seq.split("*")
          mut_seq = temp[0]
        
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])

        sequence.append(mut_seq[:2000]) # crop the sequence to 2000AA
      else:
        indices_to_excl.append(i)

Tier1_2_3 = Tier1_2_3.drop(Tier1_2_3.index[indices_to_excl])
Tier1_2_3['position'] = position
Tier1_2_3['AA_orig'] = AA_orig
Tier1_2_3['AA_targ'] = AA_targ
Tier1_2_3['sequence'] = sequence

# Keep only relevant columns 
Tier1_2_3['label'] = [1]* len(Tier1_2_3)
Tier1_2_3 = Tier1_2_3[['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'label']]
      

the original R, in position 95 not equal to: P
the original R, in position 95 not equal to: P
the original G, in position 248 not equal to: R
the original G, in position 74 not equal to: V


In [ ]:
Tier_trancript_list = Tier1_2_3['ACCESSION_NUMBER'].unique()   
# creation of benign mutations
cmcOther= missense_sil_nonsense_mut[(missense_sil_nonsense_mut['MUTATION_SIGNIFICANCE_TIER'] == 'Other')]
# keep Transcripts with Pathogenic mutations
cmcOther = cmcOther[cmcOther['ACCESSION_NUMBER'].isin(Tier_trancript_list)]

In [ ]:
cmcOther = cmcOther.sample(frac=1).reset_index(drop=True) # shuffle and reset index
count = 0
count_synonymous = 0
position, AA_orig, AA_targ, sequence, indices_to_excl = [], [], [], [], []
for i, mut in cmcOther.iterrows():
  #if count <= 18000:
    string = mut['Mutation AA']
    if '=' in string:
      count_synonymous +=1
      indices_to_excl.append(i)
      continue
    if 'delins' in string:
      indices_to_excl.append(i)
      continue
    pos_temp = int(string[3:-1])
    if pos_temp > 2000:
      indices_to_excl.append(i)
      continue
    else:
      mut_seq = changeAAnew(string[2], string[-1], pos_temp, gene_sequence[mut['ACCESSION_NUMBER']])
        
      if mut_seq is not None:
        if '*' in mut_seq:
          temp = mut_seq.split("*")
          mut_seq = temp[0]
        
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])

        sequence.append(mut_seq[:2000])
        count+=1

      else:
        indices_to_excl.append(i)


cmcOther = cmcOther.drop(cmcOther.index[indices_to_excl])
cmcOther['position'] = position
cmcOther['AA_orig'] = AA_orig
cmcOther['AA_targ'] = AA_targ
cmcOther['sequence'] = sequence

# Keep only relevant columns 
cmcOther['label'] = [0]* len(cmcOther)
cmcOther = cmcOther[['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'label']]

the original P, in position 172 not equal to: L
the original A, in position 336 not equal to: P
the original S, in position 205 not equal to: T
the original R, in position 197 not equal to: K
the original S, in position 305 not equal to: C
the original Q, in position 88 not equal to: E
the original L, in position 845 not equal to: V
the original A, in position 1136 not equal to: G
the original T, in position 1851 not equal to: R
the original L, in position 417 not equal to: H
the original L, in position 27 not equal to: I
the original P, in position 153 not equal to: R
the original G, in position 1576 not equal to: V


In [ ]:
count_synonymous # Synonymous mutations on COSMIC cmc_export dataset

127826

### Create balanced (protein) set

In [ ]:
# concatenate Tiers 1_2_3 with Other for the creation of balanced set
all_mutations = pd.concat([Tier1_2_3, cmcOther])
all_mutations = all_mutations.reset_index(drop=True) # shuffle and reset index

# label distribution per protein
lab_distr = {}
for ind, mut in all_mutations.iterrows():
  if mut['ACCESSION_NUMBER'] in lab_distr:
    if mut['label'] == 0:
      lab_distr[mut['ACCESSION_NUMBER']][0] +=1
    elif mut['label'] == 1:
      lab_distr[mut['ACCESSION_NUMBER']][1] +=1                  
  else:
    if mut['label'] == 0:
      lab_distr[mut['ACCESSION_NUMBER']] = [1, 0]
    elif mut['label'] == 1:
      lab_distr[mut['ACCESSION_NUMBER']] = [0, 1]

balanced_lab_distr = copy.deepcopy(lab_distr)
for i in balanced_lab_distr.keys():
  balanced_lab_distr[i][0] = 0

benign_to_excl = []
for i, mut in all_mutations.iterrows():

  if mut['label'] == 0:
    # have we reached the deleterious laber number of lab_distr?
    if balanced_lab_distr[mut['ACCESSION_NUMBER']][0] <= balanced_lab_distr[mut['ACCESSION_NUMBER']][1]:
      balanced_lab_distr[mut['ACCESSION_NUMBER']][0] +=1
    else:
      benign_to_excl.append(i)

all_mutations = all_mutations.drop(all_mutations.index[benign_to_excl])

In [ ]:
count_neg, count_pos = 0,0
for i, mut in all_mutations.iterrows():
  if mut['label'] == 0:
    count_neg +=1
  else:
    count_pos +=1
print(count_neg, count_pos)

18996 17639


In [ ]:
# save mutations
all_mutations.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_Other_balance_crop.csv', index=False)

In [ ]:
len(all_mutations['ACCESSION_NUMBER'].unique())

1361

In [ ]:
len(all_mutations)

36635

### For mmseq2_batch.ipynb (save Trascnrpt sequences to fasta files)

In [ ]:
# save Transcripts sequence into fasta files
alignment_path = '/content/drive/MyDrive/my_colab/3rdYear/datasets/alignments_Tier1_2_3_Other/'

for i in Tier_trancript_list:
  wt =str(gene_sequence[i])
  fasta_file = open(alignment_path + str(i) + '.fasta', 'w')
  fasta_file.write('>' + str(i) + "\n" + wt )
  fasta_file.close()



---



---



## Missense mutations form COSMIC cmc_export and CGI, with uniprot/ trembl ID + isoform info


Import - Filter


 For more info: Notes: 9 - 13/5/2022

### RUN ONCE

In [ ]:
cmc_all = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/cmc_export.csv",  delimiter = '\t') # COSMIC: cmc Tier 1,2,3, other mutations

# Filter missense / silent / nonsense mutations
missense = cmc_all[(cmc_all['Mutation Description AA'] == 'Substitution - Missense')]

# creation of Tier1-2-3 mutations
Tier1_2_3_missense = missense[(missense['MUTATION_SIGNIFICANCE_TIER'] == '1') \
                              | (missense['MUTATION_SIGNIFICANCE_TIER']== '2') \
                                       | (missense['MUTATION_SIGNIFICANCE_TIER']== '3')]

# Exclude delins
Tier1_2_3_missense = Tier1_2_3_missense.reset_index(drop=True)
delins=0
indices_to_excl = []
for i, mut in Tier1_2_3_missense.iterrows():
    string = mut['Mutation AA']
    if 'delins' in string:
      delins+=1
      indices_to_excl.append(i)

Tier1_2_3_missense = Tier1_2_3_missense.drop(Tier1_2_3_missense.index[indices_to_excl])                             

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,24,49,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Find isoforms. First compare with mart mapping. Then with mart old. If transcript found in mart_old, check if the AA orig is indead on the position of the mutation (since there is no Transcript Version on mart_old we have to be sure is the right transcript). If not found in neither mart mappings, uniprot download of isoform sequences and then keep the one that corresponds to gene_sequence from COSMIC 

In [ ]:
# Read transcript consensus (glioma_All_cosmic.ipynb)
with open("/content/drive/MyDrive/my_colab/*2ndYear/transcript_sequences.csv", 'rb') as fp:
    gene_sequence = pickle.load(fp)

In [ ]:
gene_protein_mapping = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/gene_synonyms_uniprot.txt", sep = '\t')

In [ ]:
gene_synonym_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene Synonym']]
gene_synonym_df.dropna(inplace=True)
# dropping ALL duplicate values
gene_synonym_df =gene_synonym_df.drop_duplicates(subset ='Gene Synonym')
gene_synonym_dict = gene_synonym_df.set_index('Gene Synonym').to_dict()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
gene_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene name']]
gene_df.dropna(inplace=True)
# dropping ALL duplicate values
gene_df =gene_df.drop_duplicates(subset ='Gene name')
gene_dict = gene_df.set_index('Gene name').to_dict()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
Tier1_2_3_missense = Tier1_2_3_missense.reset_index(drop=True)
transcript_not_found, delins, unknown_genes, count_test = 0, 0, 0, 0
position, AA_orig, AA_targ, sequence, indices_to_excl, canonical, isoform_uniprot = [], [], [], [], [], [], []
for i, mut in Tier1_2_3_missense.iterrows():

  gene_name = mut['GENE_NAME']
  string = mut['Mutation AA']    
  pos_temp = int(string[3:-1])
  wt_sequence = str(gene_sequence[mut['ACCESSION_NUMBER']])
  
  mut_seq = changeAAnew(gene_name, string[2], string[-1], pos_temp,wt_sequence)

  if mut_seq is not None:
    if '*' in mut_seq:
      temp = mut_seq.split("*")
      mut_seq = temp[0]
    
    if gene_name in gene_dict['UniProtKB/Swiss-Prot ID']:
      uniprot_canonical = gene_dict['UniProtKB/Swiss-Prot ID'][gene_name]
      
    elif gene_name in gene_synonym_dict['UniProtKB/Swiss-Prot ID']:
      uniprot_canonical = gene_synonym_dict['UniProtKB/Swiss-Prot ID'][gene_name] 
      
    else:
      unknown_genes +=1
      indices_to_excl.append(i)
      continue

    # load all transcript from uniprot and compare with the COSMIC transcript
    all_transcripts = load_uniprot_fasta_all(uniprot_canonical)
    count = 1
    for isoform, transcript in enumerate(all_transcripts):

      if transcript == wt_sequence:
        isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
        isoform_uniprot.append(isoform_name)
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])
        sequence.append(mut_seq) 
        break
      else:
        if len(all_transcripts) == count:
          transcript_not_found +=1
          indices_to_excl.append(i)
        count+=1
    
  else:
    indices_to_excl.append(i)


in gene SRSF2 the original R, in position 95 not equal to: P
in gene SRSF2 the original R, in position 95 not equal to: P
in gene TP53 the original G, in position 248 not equal to: R
in gene GATA1 the original G, in position 74 not equal to: V


In [ ]:
Tier1_2_3_missense = Tier1_2_3_missense.drop(Tier1_2_3_missense.index[indices_to_excl])
Tier1_2_3_missense['position'] = position
Tier1_2_3_missense['AA_orig'] = AA_orig
Tier1_2_3_missense['AA_targ'] = AA_targ
Tier1_2_3_missense['sequence'] = sequence # Mutated sequence
Tier1_2_3_missense['uniprot'] = isoform_uniprot

In [ ]:
# save mutations
Tier1_2_3_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform.csv', index=False)

In [ ]:
CGI_oncogenic_mut = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/missense_nonsense_CGI_oncogenic_mut.csv", sep = '\t')
print(len(CGI_oncogenic_mut))
print(CGI_oncogenic_mut.head())

5340
   gene                 gdna  protein       transcript  context
0  ABL1  chr9:g.133738306G>A  p.E236K  ENST00000318560  somatic
1  ABL1  chr9:g.133738309A>G  p.M237V  ENST00000318560  somatic
2  ABL1  chr9:g.133738325T>C  p.I242T  ENST00000318560  somatic
3  ABL1  chr9:g.133738330A>G  p.M244V  ENST00000318560  somatic
4  ABL1  chr9:g.133738340A>G  p.K247R  ENST00000318560  somatic


In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_mut[~CGI_oncogenic_mut['protein'].str.contains('fs')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('del')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('_')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('dup')]

In [ ]:
len(CGI_oncogenic_missense)

3782

In [ ]:
# overlap between CGI and Tier1_2_3
CGI_oncogenic_missense = CGI_oncogenic_missense.reset_index(drop=True)
position, AA_orig, AA_targ = [], [], []
for i, mut in CGI_oncogenic_missense.iterrows():
  string = mut['protein']    
  pos_temp = int(string[3:-1])

  position.append(pos_temp)
  AA_orig.append(string[2])
  AA_targ.append(string[-1])
CGI_oncogenic_missense['position'] = position
CGI_oncogenic_missense['AA_orig'] = AA_orig
CGI_oncogenic_missense['AA_targ'] = AA_targ

overlap_mut = []
for i, mut in CGI_oncogenic_missense.iterrows():
  gene = mut['gene']
  temp_df = Tier1_2_3_missense[Tier1_2_3_missense['GENE_NAME'] == gene ]
  for j, temp in temp_df.iterrows():
    if mut['position'] == temp['position'] and mut['AA_orig'] == temp['AA_orig']  and mut['AA_targ'] == temp['AA_targ'] :
      overlap_mut.append(i)

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.drop(CGI_oncogenic_missense.index[overlap_mut])

In [ ]:
len(CGI_oncogenic_missense)

2693

In [ ]:
len(CGI_oncogenic_missense[CGI_oncogenic_missense['context'] == 'somatic'])

1708

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.reset_index(drop=True)
transcript_not_found, delins, unknown_genes, count_test =0, 0, 0, 0

position, AA_orig, AA_targ, sequence, indices_to_excl, canonical, isoform_uniprot = [], [], [], [], [], [], []
for i, mut in CGI_oncogenic_missense.iterrows():

  gene_name = mut['gene']
  string = mut['protein']    
  pos_temp = int(string[3:-1])

  # exclude nonsense mutations
  if string[-1] == '*':
    indices_to_excl.append(i)
    continue

  if gene_name in gene_dict['UniProtKB/Swiss-Prot ID']:
    uniprot_canonical = gene_dict['UniProtKB/Swiss-Prot ID'][gene_name]
    
  elif gene_name in gene_synonym_dict['UniProtKB/Swiss-Prot ID']:
    uniprot_canonical = gene_synonym_dict['UniProtKB/Swiss-Prot ID'][gene_name] 
    
  else:
    unknown_genes +=1
    print(gene_name)
    indices_to_excl.append(i)
    continue

  # load all transcript from uniprot and compare with the COSMIC transcript
  all_transcripts = load_uniprot_fasta_all(uniprot_canonical)

  count = 1
  for isoform, transcript in enumerate(all_transcripts):

    if pos_temp < len(transcript):

      if transcript[pos_temp - 1] == string[2]:
        isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
        mut_seq = changeAAnew(gene_name, string[2], string[-1], pos_temp, transcript)
        isoform_uniprot.append(isoform_name)
        position.append(pos_temp)
        AA_orig.append(string[2])
        AA_targ.append(string[-1])
        sequence.append(mut_seq) 
        break
      else:
        if len(all_transcripts) == count:
          print('Different AA on position', mut, uniprot_canonical)
          transcript_not_found +=1
          indices_to_excl.append(i)
        count+=1

    else:
      if len(all_transcripts) == count:
        print('Mutations pos > length of transcript', mut, uniprot_canonical)
        transcript_not_found +=1
        indices_to_excl.append(i)
      count+=1

Different AA on position gene                              EP300
gdna          chr22:g.41564852_41564854
protein                         p.C1385
transcript              ENST00000263253
context                         somatic
position                            138
AA_orig                               C
AA_targ                               5
Name: 978, dtype: object Q09472
Different AA on position gene                              EP300
gdna          chr22:g.41572350_41572352
protein                         p.R1627
transcript              ENST00000263253
context                         somatic
position                            162
AA_orig                               R
AA_targ                               7
Name: 981, dtype: object Q09472
Different AA on position gene                         GNAS
gdna          chr20:g.57484420C>T
protein                   p.R201C
transcript        ENST00000371085
context                   somatic
position                      201
AA_orig          

KeyboardInterrupt: ignored

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.drop(CGI_oncogenic_missense.index[indices_to_excl])
CGI_oncogenic_missense['position'] = position
CGI_oncogenic_missense['AA_orig'] = AA_orig
CGI_oncogenic_missense['AA_targ'] = AA_targ
CGI_oncogenic_missense['sequence'] = sequence
CGI_oncogenic_missense['uniprot'] = isoform_uniprot

In [ ]:
# save mutations
CGI_oncogenic_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense.csv', index=False)

## Keep missense mutated sequences <= 2000 AA, save unique transcript in fasta files

In [ ]:
Tier1_2_3_missense = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform.csv", usecols = ['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'uniprot'])
CGI_oncogenic_missense = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense.csv", usecols = ['gene', 'position', 'AA_orig', 'AA_targ', 'sequence', 'uniprot'])

In [ ]:
indices_to_excl = []
Tier1_2_3_missense = Tier1_2_3_missense.reset_index(drop=True)
CGI_oncogenic_missense = CGI_oncogenic_missense.reset_index(drop=True)
for i, mut in Tier1_2_3_missense.iterrows():

  if len(mut['sequence']) > 2000:
    indices_to_excl.append(i)
Tier1_2_3_missense = Tier1_2_3_missense.drop(Tier1_2_3_missense.index[indices_to_excl])

indices_to_excl = []
for i, mut in CGI_oncogenic_missense.iterrows():

  if len(mut['sequence']) > 2000:
    indices_to_excl.append(i)
CGI_oncogenic_missense = CGI_oncogenic_missense.drop(CGI_oncogenic_missense.index[indices_to_excl])

Because of memory problem in Alphafold_batch, maximum protein length : 1500. For the (66 + 8) longer proteins, the VIBFold script in hydra was ran. Also, they were excluded by the input groups of Alphafold_batch in 3rdYear/datasets/mmseq2/ (after downloading the fasta file from datasets/fasta_Tier1_2_3_CGI_missense/)

In [ ]:
biggerthan = []
for i, mut in Tier1_2_3_missense.iterrows():

  if len(mut['sequence']) > 1500:
    biggerthan.append(mut['uniprot'])

biggerthan1 = []
for i, mut in CGI_oncogenic_missense.iterrows():

  if len(mut['sequence']) > 1500:
    biggerthan1.append(mut['uniprot'])

In [ ]:
# Find unique uniprot transcripts
Tier_trancript_list = list(Tier1_2_3_missense['uniprot'].unique())
CGI_trancript_list = list(CGI_oncogenic_missense['uniprot'].unique())
trancript_list = Tier_trancript_list + CGI_trancript_list
trancript_list = set(trancript_list)

In [ ]:
dict = {'uniprot isoform': trancript_list}  
       
df = pd.DataFrame(dict) 
    
# saving the dataframe 
df.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/uniprot_isoform2000AA.csv') 

In [ ]:
Tier1_2_3_missense['label'] = [1]* len(Tier1_2_3_missense)        
CGI_oncogenic_missense['label'] = [1]* len(CGI_oncogenic_missense)       

# find and save WT (since I didnt think about it earlier :) )
wt_sequence=[]
for i, mut in Tier1_2_3_missense.iterrows():
  wt_sequence.append(str(gene_sequence[mut['ACCESSION_NUMBER']]))
Tier1_2_3_missense['WT_sequence'] = wt_sequence

Tier1_2_3_temp = Tier1_2_3_missense[['uniprot', 'WT_sequence']]
Tier1_2_3_WTseq_dict = Tier1_2_3_temp.set_index('uniprot').to_dict()

wt_seq = []
for i, mut in CGI_oncogenic_missense.iterrows():
  mut_seq = mut['sequence']
  wt_seq.append(changeAAnew(mut['gene'], mut['AA_targ'], mut['AA_orig'], mut['position'], mut_seq))
CGI_oncogenic_missense['WT_sequence'] = wt_seq

# save mutations
Tier1_2_3_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense2000AA_isoform_WT.csv', index=False)
# save mutations
CGI_oncogenic_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense2000AA_isoform_WT.csv', index=False)

In [ ]:
CGI_temp = CGI_oncogenic_missense[['uniprot', 'WT_sequence']]
CGI_WTseq_dict = CGI_temp.set_index('uniprot').to_dict()

In [ ]:
# save Transcripts sequence into fasta files
alignment_path = '/content/drive/MyDrive/my_colab/3rdYear/datasets/fasta_Tier1_2_3_CGI_missense/'

for i in trancript_list:
  if i in Tier1_2_3_WTseq_dict['WT_sequence']:
    wt = Tier1_2_3_WTseq_dict['WT_sequence'][i]
  elif i in CGI_WTseq_dict['WT_sequence']:
    wt = CGI_WTseq_dict['WT_sequence'][i]
  fasta_file = open(alignment_path + str(i) + '.fasta', 'w')
  fasta_file.write('>' + str(i) + "\n" + wt )
  fasta_file.close()

In [ ]:
len(trancript_list)

993

In [ ]:
# save each 10 fasta sequences in different folders, ready for mmseq (AlphaFold2_batch.ipynb)
parent_dir = '/content/drive/MyDrive/my_colab/3rdYear/datasets/mmseq2/'
count = 0
for fasta_file in os.listdir(alignment_path):
  print(fasta_file)
  if not os.path.exists(parent_dir + f'input_{int(count/10)}'):
    os.mkdir(parent_dir + f'input_{int(count/10)}')
  shutil.copy(alignment_path + fasta_file, parent_dir + f'input_{int(count/10)}')
  count+=1

A4D1F6-1.fasta
O75475-1.fasta
P04233-1.fasta
P30281-1.fasta
Q5SRI9-1.fasta
Q86UR5-1.fasta
P36897-1.fasta
Q9ULS5-1.fasta
Q96SL8-1.fasta
Q32MK0-1.fasta
Q8N136-1.fasta
Q92889-1.fasta
Q8NEK8-1.fasta
O00255-1.fasta
O75344-1.fasta
Q3LI73-1.fasta
O95302-1.fasta
Q96A98-1.fasta
P24347-1.fasta
Q2M329-1.fasta
P32243-2.fasta
Q8WV37-1.fasta
Q9P2K8-1.fasta
P54803-1.fasta
Q6ZMC9-1.fasta
P00519-1.fasta
Q9NYA1-2.fasta
P31314-1.fasta
Q2TAZ0-1.fasta
P05141-1.fasta
P21912-1.fasta
Q8NG06-1.fasta
Q9NX18-1.fasta
P37173-2.fasta
Q9BYR4-1.fasta
P29508-1.fasta
Q9Y5Y0-1.fasta
P14210-1.fasta
P05412-1.fasta
Q9BXJ2-2.fasta
Q9UHL4-1.fasta
Q9H7D0-1.fasta
Q5VTJ3-1.fasta
Q9BTP6-1.fasta
Q8N7X4-1.fasta
Q9BXL7-1.fasta
Q5T124-1.fasta
Q8N5M9-1.fasta
Q9UMS4-1.fasta
O15069-1.fasta
O43791-1.fasta
Q2TAM9-1.fasta
Q00888-1.fasta
P05556-1.fasta
P07492-1.fasta
Q15047-1.fasta
Q99612-1.fasta
O43166-1.fasta
Q9UHD2-1.fasta
Q6UUV9-2.fasta
Q9UHC6-1.fasta
O60674-1.fasta
P16871-1.fasta
Q6BEB4-1.fasta
Q16832-1.fasta
P07949-1.fasta
Q9UKQ2-1.f

## Benign mutations for Uniprot ID that I have for deleterious

Find how many deleterious mutations I have per uniprot id. Add the same amount of mutations from humsavar

In [ ]:
humsavar = pd.read_csv('/content/drive/MyDrive/my_colab/*2ndYear/humsavar_1221_neut.csv')

trancript_list = list(trancript_list)
wt_sequences, mut_sequences, uniprot_list, AA_orig_list, AA_targ_list, position_list = [], [], [], [], [], []
for uniprot in trancript_list:

  tier = Tier1_2_3_missense[Tier1_2_3_missense['uniprot'] == uniprot]
  CGI = CGI_oncogenic_missense[CGI_oncogenic_missense['uniprot'] == uniprot]
  if len(tier) > 0 :
    wt_sequence = tier.iloc[0]['WT_sequence']
  else:
    wt_sequence = CGI.iloc[0]['WT_sequence']
  nb_tier = len(tier)
  nb_CGI= len(CGI)
  nb_del = nb_tier + nb_CGI

  temp= humsavar[humsavar['Protein'] == uniprot[:-2]]

  for i, mut in temp.iterrows():
    AA_orig = AAconvert(mut['Mutation'][2:5])
    AA_targ = AAconvert(mut['Mutation'][-3:])
    position = int(mut['Mutation'][5:-3])
    if position <= len(wt_sequence):
      if wt_sequence[position - 1] == AA_orig:
        wt_sequences.append(wt_sequence)
        mut_sequences.append(changeAAnew(uniprot, AA_orig, AA_targ, position, wt_sequence))
        uniprot_list.append(uniprot)
        AA_orig_list.append(AA_orig)
        AA_targ_list.append(AA_targ)
        position_list.append(position)

# select 2000 benign more that dont exist in Tier or CGI Missense

for i, mut in humsavar.iterrows():
  if i >1500:
    break
  if mut['Protein'] not in trancript_list:
        wt_sequences.append(wt_sequence)
        mut_sequences.append(changeAAnew(uniprot, AA_orig, AA_targ, position, wt_sequence))
        uniprot_list.append(uniprot)
        AA_orig_list.append(AA_orig)
        AA_targ_list.append(AA_targ)
        position_list.append(position)


benign = pd.DataFrame(list(zip(uniprot_list, wt_sequences, mut_sequences, AA_orig_list, position_list, AA_targ_list)),
               columns =['uniprot id', 'WT_sequence', 'mut_sequence', 'A_orig_list', 'position_list', 'AA_targ_list'])


benign['label'] = [0]* len(benign)      

In [ ]:
len(benign)

5301

In [ ]:
# save mutations
benign.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/humsavar2000AA_isoform_WT.csv', index=False)

## Analysis of mutations present in missense Dataset

In [ ]:
Tier1_2_3_missense = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform.csv", usecols = ['GENE_NAME', 'ACCESSION_NUMBER', 'position', 'AA_orig', 'AA_targ', 'sequence', 'uniprot'])
CGI_oncogenic_missense = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense.csv", usecols = ['gene', 'position', 'AA_orig', 'AA_targ', 'sequence', 'uniprot'])

In [ ]:
list_uniprot_Tier1_2_3_missense = Tier1_2_3_missense['uniprot']
list_uniprot_CGI_oncogenic_missense = CGI_oncogenic_missense['uniprot']

In [ ]:
for i in list_uniprot_Tier1_2_3_missense:
  if i in list_uniprot_CGI_oncogenic_missense:
    print(i)

In [ ]:
CGI_oncogenic_mut = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/missense_nonsense_CGI_oncogenic_mut.csv", sep = '\t')

In [ ]:
CGI_oncogenic_missense

,gene,gdna,protein,transcript,context,position,AA_orig,AA_targ
0,ABL1,chr9:g.133738306G>A,p.E236K,ENST00000318560,somatic,236,E,K
1,ABL1,chr9:g.133738309A>G,p.M237V,ENST00000318560,somatic,237,M,V
2,ABL1,chr9:g.133738325T>C,p.I242T,ENST00000318560,somatic,242,I,T
3,ABL1,chr9:g.133738330A>G,p.M244V,ENST00000318560,somatic,244,M,V
4,ABL1,chr9:g.133738340A>G,p.K247R,ENST00000318560,somatic,247,K,R
...,...,...,...,...,...,...,...,...
3776,WHSC1,chr4:g.1962801G>A,p.E1099K,ENST00000382895,somatic,1099,E,K
3777,WRN,chr8:g.30924605A>G,p.K187K,ENST00000298139,germline,187,K,K
3778,WWOX,chr16:g.78466465T>C,p.L291P,ENST00000566780,somatic,291,L,P
3779,XPC,chr3:g.14199706G>T,p.Y559*,ENST00000285021,germline,559,Y,*


## [ New version] Missense mutations form COSMIC cmc_export and CGI, with uniprot/ trembl ID + isoform info  - includes multiple uniprot ids per gene- excludes germline mutations CGI. Bening test from humsavar

In [ ]:
cmc_all = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/cmc_export.csv",  delimiter = '\t', usecols = ['GENE_NAME', 'ACCESSION_NUMBER', 'Mutation AA', 'Mutation Description AA', 'MUTATION_SIGNIFICANCE_TIER']) # COSMIC: cmc Tier 1,2,3, other mutations

# Filter out silent / nonsense mutations
missense = cmc_all[(cmc_all['Mutation Description AA'] == 'Substitution - Missense')]

# creation of Tier1-2-3 mutations
Tier1_2_3_missense = missense[(missense['MUTATION_SIGNIFICANCE_TIER'] == '1') \
                              | (missense['MUTATION_SIGNIFICANCE_TIER']== '2') \
                                       | (missense['MUTATION_SIGNIFICANCE_TIER']== '3')]
# Exclude delins
Tier1_2_3_missense = Tier1_2_3_missense.reset_index(drop=True)
delins=0
indices_to_excl = []
for i, mut in Tier1_2_3_missense.iterrows():
    string = mut['Mutation AA']
    if 'delins' in string:
      delins+=1
      indices_to_excl.append(i)

Tier1_2_3_missense = Tier1_2_3_missense.drop(Tier1_2_3_missense.index[indices_to_excl])                             

In [ ]:
len(Tier1_2_3_missense)

3929

In [ ]:
# Read transcript consensus (glioma_All_cosmic.ipynb)
with open("/content/drive/MyDrive/my_colab/*2ndYear/transcript_sequences.csv", 'rb') as fp:
    gene_sequence = pickle.load(fp)

gene_protein_mapping = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/gene_synonyms_uniprot.txt", sep = '\t')
# Synonymous genes
gene_synonym_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene Synonym']]
gene_synonym_df.dropna(inplace=True)
gene_synonym_df =gene_synonym_df.drop_duplicates()
gene_synonym_dict = gene_synonym_df.groupby('Gene Synonym')['UniProtKB/Swiss-Prot ID'].apply(lambda g: g.values.tolist()).to_dict()

gene_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene name']]
gene_df.dropna(inplace=True)
# dropping ALL duplicate values
gene_df =gene_df.drop_duplicates()
gene_dict = gene_df.groupby('Gene name')['UniProtKB/Swiss-Prot ID'].apply(lambda g: g.values.tolist()).to_dict()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
Tier1_2_3_missense = Tier1_2_3_missense.reset_index(drop=True)
transcript_not_found, delins, unknown_genes, count_test = 0, 0, 0, 0
position, AA_orig, AA_targ, sequence, indices_to_excl, canonical, isoform_uniprot = [], [], [], [], [], [], []
count_found, count_not_found = 0, 0
for i, mut in Tier1_2_3_missense.iterrows():

  gene_name = mut['GENE_NAME']
  string = mut['Mutation AA']    
  pos_temp = int(string[3:-1])
  wt_sequence = str(gene_sequence[mut['ACCESSION_NUMBER']])
  
  mut_seq = changeAAnew(mut['ACCESSION_NUMBER'], string[2], string[-1], pos_temp,wt_sequence)

  if mut_seq is not None:
    if '*' in mut_seq:
      temp = mut_seq.split("*")
      mut_seq = temp[0]
    
    if gene_name in gene_dict:
      uniprot_canonical_list = gene_dict[gene_name]# we may have more than one corresponding proteins for gene
    
    elif gene_name in gene_synonym_dict:
      uniprot_canonical_list = gene_synonym_dict[gene_name] 

    else:
      unknown_genes +=1
      indices_to_excl.append(i)
      continue
    
    count_uniprot =0 # track isoforms
    found=False # used to stop searching follwing isoforms

    for uniprot_canonical in uniprot_canonical_list:
      count_uniprot+=1 
      if found == True:
        break

      # load all transcript from uniprot and compare with the COSMIC transcript
      all_transcripts = load_uniprot_fasta_all(uniprot_canonical)
      count = 1 # track transcripts
      for isoform, transcript in enumerate(all_transcripts):

        if transcript == wt_sequence:
          isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
          isoform_uniprot.append(isoform_name)
          position.append(pos_temp)
          AA_orig.append(string[2])
          count_found +=1
          AA_targ.append(string[-1])
          found = True
          sequence.append(mut_seq) 
          break
        else:
          if len(all_transcripts) == count and count_uniprot == len(uniprot_canonical_list):
            transcript_not_found +=1
            indices_to_excl.append(i)
          count+=1
  else:
    indices_to_excl.append(i)
    count_not_found+=1

in gene ENST00000392485.2 the original R, in position 95 not equal to: P
in gene ENST00000392485.2 the original R, in position 95 not equal to: P
in gene ENST00000269305.4 the original G, in position 248 not equal to: R
in gene ENST00000376670.3 the original G, in position 74 not equal to: V


In [ ]:
Tier1_2_3_missense = Tier1_2_3_missense.drop(Tier1_2_3_missense.index[indices_to_excl])
Tier1_2_3_missense['position'] = position
Tier1_2_3_missense['AA_orig'] = AA_orig
Tier1_2_3_missense['AA_targ'] = AA_targ
Tier1_2_3_missense['mut_sequence'] = sequence # Mutated sequence
Tier1_2_3_missense['uniprot'] = isoform_uniprot

In [ ]:
# save mutations
Tier1_2_3_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform_2.csv', index=False)

In [ ]:
CGI_oncogenic_mut = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/missense_nonsense_CGI_oncogenic_mut.csv", sep = '\t')
print(len(CGI_oncogenic_mut))
print(CGI_oncogenic_mut.head())

5340
   gene                 gdna  protein       transcript  context
0  ABL1  chr9:g.133738306G>A  p.E236K  ENST00000318560  somatic
1  ABL1  chr9:g.133738309A>G  p.M237V  ENST00000318560  somatic
2  ABL1  chr9:g.133738325T>C  p.I242T  ENST00000318560  somatic
3  ABL1  chr9:g.133738330A>G  p.M244V  ENST00000318560  somatic
4  ABL1  chr9:g.133738340A>G  p.K247R  ENST00000318560  somatic


In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_mut[~CGI_oncogenic_mut['protein'].str.contains('fs')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('del')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('_')]
CGI_oncogenic_missense = CGI_oncogenic_missense[~CGI_oncogenic_missense['protein'].str.contains('dup')]

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense[CGI_oncogenic_missense['context'] == 'somatic']

In [ ]:
# overlap between CGI and Tier1_2_3
CGI_oncogenic_missense = CGI_oncogenic_missense.reset_index(drop=True)
position, AA_orig, AA_targ = [], [], []
for i, mut in CGI_oncogenic_missense.iterrows():
  string = mut['protein']    
  pos_temp = int(string[3:-1])

  position.append(pos_temp)
  AA_orig.append(string[2])
  AA_targ.append(string[-1])
CGI_oncogenic_missense['position'] = position
CGI_oncogenic_missense['AA_orig'] = AA_orig
CGI_oncogenic_missense['AA_targ'] = AA_targ

overlap_mut = []
for i, mut in CGI_oncogenic_missense.iterrows():
  gene = mut['gene']
  temp_df = Tier1_2_3_missense[Tier1_2_3_missense['GENE_NAME'] == gene ]
  for j, temp in temp_df.iterrows():
    if mut['position'] == temp['position'] and mut['AA_orig'] == temp['AA_orig']  and mut['AA_targ'] == temp['AA_targ'] :
      overlap_mut.append(i)

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.drop(CGI_oncogenic_missense.index[overlap_mut])

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.reset_index(drop=True)
transcript_not_found, delins, unknown_genes, count_test = 0, 0, 0, 0
position, AA_orig, AA_targ, sequence, indices_to_excl, canonical, isoform_uniprot = [], [], [], [], [], [], []
count_found, count_not_found = 0, 0
for i, mut in CGI_oncogenic_missense.iterrows():

  gene_name = mut['gene']
  string = mut['protein']    
  pos_temp = int(string[3:-1])

  # exclude nonsense mutations
  if string[-1] == '*':
    indices_to_excl.append(i)
    continue

  if '*' in mut_seq:
    temp = mut_seq.split("*")
    mut_seq = temp[0]
  
  if gene_name in gene_dict:
    uniprot_canonical_list = gene_dict[gene_name] # we may have more than one corresponding proteins for gene
  
  elif gene_name in gene_synonym_dict:
    uniprot_canonical_list = gene_synonym_dict[gene_name] 

  else:
    unknown_genes +=1
    indices_to_excl.append(i)
    continue
  
  count_uniprot =0 # track isoforms
  found=False # used to stop searching follwing isoforms

  for uniprot_canonical in uniprot_canonical_list:
    count_uniprot+=1 
    if found == True:
      break

    # load all transcript from uniprot
    all_transcripts = load_uniprot_fasta_all(uniprot_canonical)
    count = 1 # track transcripts
    for isoform, transcript in enumerate(all_transcripts):

      if pos_temp <= len(transcript):

          if transcript[pos_temp - 1] == string[2]:
            isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
            mut_seq = changeAAnew(gene_name, string[2], string[-1], pos_temp, transcript)
            isoform_uniprot.append(isoform_name)
            position.append(pos_temp)
            AA_orig.append(string[2])
            AA_targ.append(string[-1])
            count_found +=1
            found = True
            sequence.append(mut_seq) 
            break

          else:
            if len(all_transcripts) == count and count_uniprot == len(uniprot_canonical_list):
              transcript_not_found +=1
              indices_to_excl.append(i)
            count+=1

      else:
          if len(all_transcripts) == count and count_uniprot == len(uniprot_canonical_list):
            transcript_not_found +=1
            indices_to_excl.append(i)
          count+=1

In [ ]:
CGI_oncogenic_missense = CGI_oncogenic_missense.drop(CGI_oncogenic_missense.index[indices_to_excl])
CGI_oncogenic_missense['position'] = position
CGI_oncogenic_missense['AA_orig'] = AA_orig
CGI_oncogenic_missense['AA_targ'] = AA_targ
CGI_oncogenic_missense['sequence'] = sequence
CGI_oncogenic_missense['uniprot'] = isoform_uniprot

In [ ]:
# save mutations
CGI_oncogenic_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense_2.csv', index=False)

In [ ]:
# Find unique uniprot transcripts
Tier_trancript_list = list(Tier1_2_3_missense['uniprot'].unique())
CGI_trancript_list = list(CGI_oncogenic_missense['uniprot'].unique())
trancript_list = Tier_trancript_list + CGI_trancript_list
trancript_list = set(trancript_list)

NameError: ignored

In [ ]:
Tier1_2_3_missense['label'] = [1]* len(Tier1_2_3_missense)        
CGI_oncogenic_missense['label'] = [1]* len(CGI_oncogenic_missense)       

# find and save WT (since I didnt think about it earlier :) )
wt_sequence=[]
for i, mut in Tier1_2_3_missense.iterrows():
  wt_sequence.append(str(gene_sequence[mut['ACCESSION_NUMBER']]))
Tier1_2_3_missense['WT_sequence'] = wt_sequence

Tier1_2_3_temp = Tier1_2_3_missense[['uniprot', 'WT_sequence']]
Tier1_2_3_WTseq_dict = Tier1_2_3_temp.set_index('uniprot').to_dict()

wt_seq = []
for i, mut in CGI_oncogenic_missense.iterrows():
  mut_seq = mut['sequence']
  wt_seq.append(changeAAnew(mut['gene'], mut['AA_targ'], mut['AA_orig'], mut['position'], mut_seq))
CGI_oncogenic_missense['WT_sequence'] = wt_seq

# save mutations
Tier1_2_3_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform_2WT.csv', index=False)
# save mutations
CGI_oncogenic_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense_isoform_2WT.csv', index=False)

In [ ]:
# to delete
Tier1_2_3_missense =pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform_2WT.csv')
CGI_oncogenic_missense = pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense_isoform_2WT.csv')

# Find unique uniprot transcripts
Tier_trancript_list = list(Tier1_2_3_missense['uniprot'].unique())
CGI_trancript_list = list(CGI_oncogenic_missense['uniprot'].unique())
trancript_list = Tier_trancript_list + CGI_trancript_list
trancript_list = set(trancript_list)

In [ ]:
len(Tier1_2_3_missense) + len(CGI_oncogenic_missense)

5296

Choose benign proteins with at least one deleterious mutations Careful - there are humsavar benigns that are on Tier1_2_3 (295)

In [ ]:
humsavar = pd.read_csv('/content/drive/MyDrive/my_colab/*2ndYear/humsavar_1221_neut.csv')

trancript_list = list(trancript_list)
sub='-1'
canonical_list = [s for s in trancript_list if sub in s]

wt_sequences, mut_sequences, uniprot_list, AA_orig_list, AA_targ_list, position_list = [], [], [], [], [], []
for uniprot in canonical_list:

  tier = Tier1_2_3_missense[Tier1_2_3_missense['uniprot'] == uniprot]
  CGI = CGI_oncogenic_missense[CGI_oncogenic_missense['uniprot'] == uniprot]
  if len(tier) > 0 :
    wt_sequence = tier.iloc[0]['WT_sequence']
  else:
    wt_sequence = CGI.iloc[0]['WT_sequence']
  nb_tier = len(tier)
  nb_CGI= len(CGI)
  nb_del = nb_tier + nb_CGI

  if nb_del >0:

    temp= humsavar[humsavar['Protein'] == uniprot[:-2]]

    for i, mut in temp.iterrows():
      
      AA_orig = AAconvert(mut['Mutation'][2:5])
      AA_targ = AAconvert(mut['Mutation'][-3:])
      position = int(mut['Mutation'][5:-3])
      if position <= len(wt_sequence):
        if wt_sequence[position - 1] == AA_orig:
          wt_sequences.append(wt_sequence)
          mut_sequences.append(changeAAnew(uniprot, AA_orig, AA_targ, position, wt_sequence))
          uniprot_list.append(uniprot)
          AA_orig_list.append(AA_orig)
          AA_targ_list.append(AA_targ)
          position_list.append(position)

In [ ]:
len(position_list)

4432

In [ ]:
len(trancript_list)

1143

In [ ]:
humsavar = pd.read_csv('/content/drive/MyDrive/my_colab/*2ndYear/humsavar_1221_neut.csv')

trancript_list = list(trancript_list)
test, test2 =0,0
wt_sequences, mut_sequences, uniprot_list, AA_orig_list, AA_targ_list, position, indices_to_excl = [], [], [], [], [], [], []
for uniprot in trancript_list:

  tier = Tier1_2_3_missense[Tier1_2_3_missense['uniprot'] == uniprot] # how many deleterious mutations I have for uniprot id
  CGI = CGI_oncogenic_missense[CGI_oncogenic_missense['uniprot'] == uniprot]
  #if len(tier) > 0 :
  #  wt_sequence = tier.iloc[0]['WT_sequence']
  #else:
  #  wt_sequence = CGI.iloc[0]['WT_sequence']
  nb_tier = len(tier)
  nb_CGI= len(CGI)
  nb_del = nb_tier + nb_CGI
  if nb_del >0:

    temp= humsavar[humsavar['Protein'] == uniprot[:-2]]

    for i, mut in temp.iterrows():
      AA_orig = AAconvert(mut['Mutation'][2:5])
      AA_targ = AAconvert(mut['Mutation'][-3:])
      pos_temp = int(mut['Mutation'][5:-3])
          
      # load all transcript from uniprot
      all_transcripts = load_uniprot_fasta_all(mut['Protein'])
      #transcript= all_transcripts[0] # for canonical
      #isoform=0 # for canonical
      count = 1 # track transcripts
      for isoform, transcript in enumerate(all_transcripts):

        if pos_temp <= len(transcript):

            if transcript[pos_temp - 1] == AA_orig:
              isoform_name = mut['Protein']+ '-' + str(isoform+1) 
              mut_seq = changeAAnew(mut['Protein'], AA_orig, AA_targ, pos_temp, transcript)
              uniprot_list.append(isoform_name)
              position.append(pos_temp)
              AA_orig_list.append(AA_orig)
              AA_targ_list.append(AA_targ)
              mut_sequences.append(mut_seq) 
              wt_sequences.append(transcript)
              break

            else:
              if len(all_transcripts) == count:
                test+=1
              count+=1

        else:
            if len(all_transcripts) == count:
              test2+=1
            count+=1

In [ ]:
print(len(position))

4960


In [ ]:
# select 150 benign more that dont exist in Tier or CGI Missense
test, test2 = 0,0
for i, mut in humsavar.iterrows():
  if i >750:
    break

  #if mut['Protein'] + '-1' not in canonical_list:
  if mut['Protein'] + '-1' not in trancript_list:
    
    AA_orig = AAconvert(mut['Mutation'][2:5])
    AA_targ = AAconvert(mut['Mutation'][-3:])
    pos_temp = int(mut['Mutation'][5:-3])

    # load all transcript from uniprot
    all_transcripts = load_uniprot_fasta_all(mut['Protein'])
    transcript= all_transcripts[0]
    count = 1 # track transcripts
    isoform=0
    for isoform, transcript in enumerate(all_transcripts):
      if pos_temp <= len(transcript):

          if transcript[pos_temp - 1] == AA_orig:
            isoform_name = mut['Protein']+ '-' + str(isoform+1) 
            mut_seq = changeAAnew(mut['Protein'], AA_orig, AA_targ, pos_temp, transcript)
            uniprot_list.append(isoform_name)
            position.append(pos_temp)
            AA_orig_list.append(AA_orig)
            AA_targ_list.append(AA_targ)
            mut_sequences.append(mut_seq) 
            wt_sequences.append(transcript)
            break

          else:
            if len(all_transcripts) == count:
              test+=1
            count+=1

      else:
          if len(all_transcripts) == count:
            test2+=1
          count+=1

In [ ]:
len(wt_sequences)

5664

In [ ]:
benign = pd.DataFrame(list(zip(uniprot_list, wt_sequences, mut_sequences, AA_orig_list, position, AA_targ_list)),
               columns =['uniprot id', 'WT_sequence', 'mut_sequence', 'A_orig_list', 'position', 'AA_targ_list'])
benign['label'] = [0]* len(benign) 
benign.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/humsavar_isoform_5WT.csv', index=False)

## [NEW VERSION] Tier1_2_3_Other, balanced with regard to protein uniprot id

In [ ]:
cmc_all = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/cmc_export.csv",  delimiter = '\t', usecols = ['GENE_NAME', 'ACCESSION_NUMBER', 'Mutation AA', 'Mutation Description AA', 'MUTATION_SIGNIFICANCE_TIER']) # COSMIC: cmc Tier 1,2,3, other mutations

# Filter out silent / nonsense mutations
missense = cmc_all[(cmc_all['Mutation Description AA'] == 'Substitution - Missense')]

# creation of Other mutations
Other_missense = missense[(missense['MUTATION_SIGNIFICANCE_TIER'] == 'Other') ]
# Exclude delins
Other_missense = Other_missense.reset_index(drop=True)
delins=0
indices_to_excl = []
for i, mut in Other_missense.iterrows():
    string = mut['Mutation AA']
    if 'delins' in string or 'fs' in string:
      delins+=1
      indices_to_excl.append(i)

Other_missense = Other_missense.drop(Other_missense.index[indices_to_excl])      

In [ ]:
Tier_accession_list = list(Tier1_2_3_missense['ACCESSION_NUMBER'].unique())   
CGI_gene_list =  list(CGI_oncogenic_missense['gene'].unique())
# creation of benign mutations
# keep Transcripts with Tier1_2_3 Pathogenic mutations (Other with Uniprot isoform in Tier1_2_3)
Other_missense1 = Other_missense[Other_missense['ACCESSION_NUMBER'].isin(Tier_accession_list)]
# keep mutations on genes that have a CGI mutation
Other_missense2 = Other_missense[Other_missense['GENE_NAME'].isin(CGI_gene_list)]

In [ ]:
uniprot, mut_sequences,wt_sequences, ind_exc, AA_orig, AA_targ, position  =[], [],[], [], [], [], []
Other_missense1 = Other_missense1.reset_index(drop=True)
for i, mut in Other_missense1.iterrows():
   string = mut['Mutation AA']
   pos = int(string[3:-1])
   AA_o = string[2]
   AA_t = string[-1]
   accec_nb = mut['ACCESSION_NUMBER']
   t = Tier1_2_3_missense.loc[Tier1_2_3_missense['ACCESSION_NUMBER'] == accec_nb]
   uniprot.append(t.iloc[0]['uniprot'])
   wtseq = t.iloc[0]['WT_sequence']
   mut_seq = changeAAnew(accec_nb, AA_o, AA_t, pos, wtseq)
   if mut_seq == None:
     ind_exc.append(i)
     continue
   mut_sequences.append(mut_seq) 
   wt_sequences.append(wtseq)
   AA_orig.append(AA_o)
   AA_targ.append(AA_t)
   position.append(pos)

Other_missense1['uniprot'] = uniprot
Other_missense1 = Other_missense1.drop(Other_missense1.index[ind_exc])   #exclude errors in cmc

Other_missense1['mut_sequence'] = mut_sequences
Other_missense1['WT_sequence'] = wt_sequences
Other_missense1['AA_orig'] = AA_orig
Other_missense1['AA_targ'] = AA_targ
Other_missense1['position'] = position

in gene ENST00000280772.2 the original A, in position 3507 not equal to: G
in gene ENST00000397910.4 the original S, in position 14068 not equal to: T
in gene ENST00000397910.4 the original W, in position 13862 not equal to: C
in gene ENST00000397910.4 the original M, in position 14083 not equal to: I
in gene ENST00000396930.3 the original P, in position 153 not equal to: R
in gene ENST00000262189.6 the original R, in position 197 not equal to: K
in gene ENST00000373644.4 the original P, in position 172 not equal to: L
in gene ENST00000367202.4 the original S, in position 205 not equal to: T
in gene ENST00000343736.4 the original T, in position 1851 not equal to: R
in gene ENST00000457416.2 the original L, in position 417 not equal to: H
in gene ENST00000356352.2 the original L, in position 845 not equal to: V
in gene ENST00000498124.1 the original Q, in position 88 not equal to: E
in gene ENST00000277541.6 the original G, in position 1576 not equal to: V
in gene ENST00000367409.4 the 

In [ ]:
# from Other_missense2 confirm there are the right uniprot ids :
Other_missense2 = Other_missense2.reset_index(drop=True)
ind_exc, uniprot, mut_sequences, wt_sequences, AA_orig, AA_targ, position  =[], [],[], [], [], [], []
for i, mut in Other_missense2.iterrows():
  gene = mut['GENE_NAME']
  string = mut['Mutation AA']
  pos = int(string[3:-1])
  AA_o = string[2]
  AA_t = string[-1]
  t = CGI_oncogenic_missense.loc[CGI_oncogenic_missense['gene'] == gene]
  wtseq = t.iloc[0]['WT_sequence']
  uniprot.append(t.iloc[0]['uniprot'])
  if len(wtseq) < pos:
    ind_exc.append(i)
    continue
  elif wtseq[pos - 1] != AA_o:
    ind_exc.append(i)
    continue

  mut_seq = changeAAnew(gene, AA_o, AA_t, pos, wtseq)
  if mut_seq is not None:
    mut_sequences.append(mut_seq) 
    wt_sequences.append(wtseq)
    AA_orig.append(AA_o)
    AA_targ.append(AA_t)
    position.append(pos)
  else:
    ind_exc.append(i)

Other_missense2['uniprot'] = uniprot

Other_missense2 = Other_missense2.drop(Other_missense2.index[ind_exc])   #exclude errors in cmc
Other_missense2['mut_sequence'] = mut_sequences
Other_missense2['WT_sequence'] =wt_sequences
Other_missense2['AA_orig'] = AA_orig
Other_missense2['AA_targ'] = AA_targ
Other_missense2['position'] = position

In [ ]:
all_others = pd.concat([Other_missense1, Other_missense2]) # join 2 df
all_others = all_others.reset_index(drop=True)

all_others['label'] = [0]* len(all_others)     
all_others_reorder = all_others[['uniprot', 'mut_sequence', 'WT_sequence', 'AA_orig', 'position', 'AA_targ', 'label']]

In [ ]:
# load deleterious mutations
Tier1_2_3_missense= pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_missense_isoform_2WT.csv', usecols = ['uniprot', 'mut_sequence', 'WT_sequence', 'AA_orig',	'position', 'AA_targ' ,'label']) # /Tier1_2_3_missense2000AA_isoform_WT.csv
# s# load CGI mutations
CGI_oncogenic_missense= pd.read_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CGI_oncogenic_missense_isoform_2WT.csv', usecols = ['uniprot', 'sequence', 'WT_sequence', 'AA_orig',	'position', 'AA_targ' ,'label']) # CGI_oncogenic_missense2000AA_isoform_WT


CGI_oncogenic_missense.rename(columns= {'sequence': 'mut_sequence'}, inplace=True)
CGI_oncogenic_reorder = CGI_oncogenic_missense[['uniprot', 'mut_sequence', 'WT_sequence', 'AA_orig', 'position', 'AA_targ', 'label']]
Tier1_2_3_missense_reorder = Tier1_2_3_missense[['uniprot', 'mut_sequence', 'WT_sequence', 'AA_orig', 'position', 'AA_targ', 'label']]

# merge dataframes
all_Tiers = pd.concat([CGI_oncogenic_reorder, Tier1_2_3_missense_reorder ])

all_Tiers = all_Tiers.reset_index(drop=True)
all_Tiers = all_Tiers.drop_duplicates()
all_Tiers = all_Tiers.sort_values(by='label', ascending=False)
all_Tiers = all_Tiers.drop_duplicates(subset=['uniprot','position','AA_targ' ], keep='first')

In [ ]:
# concatenate Tiers 1_2_3 with Other for the creation of balanced set
all_mutations = pd.concat([all_Tiers, all_others_reorder])
all_mutations = all_mutations.reset_index(drop=True) # shuffle and reset index

# label distribution per protein
lab_distr = {}
for ind, mut in all_mutations.iterrows():
  if mut['uniprot'] in lab_distr:
    if mut['label'] == 0:
      lab_distr[mut['uniprot']][0] +=1
    elif mut['label'] == 1:
      lab_distr[mut['uniprot']][1] +=1                  
  else:
    if mut['label'] == 0:
      lab_distr[mut['uniprot']] = [1, 0]
    elif mut['label'] == 1:
      lab_distr[mut['uniprot']] = [0, 1]

balanced_lab_distr = copy.deepcopy(lab_distr)
for i in balanced_lab_distr.keys():
  balanced_lab_distr[i][0] = 0

benign_to_excl = []
for i, mut in all_mutations.iterrows():

  if mut['label'] == 0:
    # have we reached the deleterious laber number of lab_distr?
    if balanced_lab_distr[mut['uniprot']][0] < balanced_lab_distr[mut['uniprot']][1]:
      balanced_lab_distr[mut['uniprot']][0] +=1
    else:
      benign_to_excl.append(i)

all_mutations = all_mutations.drop(all_mutations.index[benign_to_excl])

In [ ]:
count_neg, count_pos = 0,0
for i, mut in all_mutations.iterrows():
  if mut['label'] == 0:
    count_neg +=1
  else:
    count_pos +=1
print(count_neg, count_pos)

5039 5134


In [ ]:
all_mutations.head()

,uniprot,mut_sequence,WT_sequence,AA_orig,position,AA_targ,label
0,P00519-1,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,E,236,K,1
1,Q15059-1,MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVV...,MSTATTVAPAGIPATPGPVNPPPPEVSNPSKPGRKTNQLQYMQNVV...,H,74,Y,1
2,P29992-1,MTLESMMACCLSDEVKESKRINAEIEKQLRRDKRDARRELKLLLLG...,MTLESMMACCLSDEVKESKRINAEIEKQLRRDKRDARRELKLLLLG...,Q,209,P,1
3,P29992-1,MTLESMMACCLSDEVKESKRINAEIEKQLRRDKRDARRELKLLLLG...,MTLESMMACCLSDEVKESKRINAEIEKQLRRDKRDARRELKLLLLG...,Q,209,L,1
4,P53621-2,MLTKFETKSARVKGLSFHPKRPWILTSLHNGVIQLWDYRMCTLIDK...,MLTKFETKSARVKGLSFHPKRPWILTSLHNGVIQLWDYRMCTLIDK...,T,815,P,1


In [ ]:
all_mutations.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/Tier1_2_3_CGI_Other.csv', index=False) # There are uniprots with Tier but not Other mutations

## Clinvar missense germline

In [ ]:
# Read transcript consensus (glioma_All_cosmic.ipynb)
with open("/content/drive/MyDrive/my_colab/*2ndYear/transcript_sequences.csv", 'rb') as fp:
    gene_sequence = pickle.load(fp)

gene_protein_mapping = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/gene_synonyms_uniprot.txt", sep = '\t')
# Synonymous genes
gene_synonym_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene Synonym']]
gene_synonym_df.dropna(inplace=True)
gene_synonym_df =gene_synonym_df.drop_duplicates()
gene_synonym_dict = gene_synonym_df.groupby('Gene Synonym')['UniProtKB/Swiss-Prot ID'].apply(lambda g: g.values.tolist()).to_dict()

gene_df = gene_protein_mapping[['UniProtKB/Swiss-Prot ID', 'Gene name']]
gene_df.dropna(inplace=True)
# dropping ALL duplicate values
gene_df =gene_df.drop_duplicates()
gene_dict = gene_df.groupby('Gene name')['UniProtKB/Swiss-Prot ID'].apply(lambda g: g.values.tolist()).to_dict()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
path = "/content/drive/MyDrive/my_colab/*2ndYear/foldx/"
filename = "variant_summary0222.txt"
clinvar = pd.read_csv(path + filename, sep = '\t', usecols = ['Type', 'Name', 'GeneSymbol', 'ClinicalSignificance', 'OriginSimple', 'ReviewStatus']) #'Assembly', 'ClinSigSimple',

clinvar = clinvar.drop_duplicates()

KeyboardInterrupt: ignored

In [ ]:
len(clinvar)

1193431

In [ ]:
len(clinvar_VUS)

498890

In [ ]:
clinvar_VUS= clinvar.loc[(clinvar['ClinicalSignificance'] == 'Uncertain significance') \
                              | (clinvar['ClinicalSignificance'] == 'Conflicting interpretations of pathogenicity, other, risk factor') \
                              | (clinvar['ClinicalSignificance'] == 'Conflicting interpretations of pathogenicity, other') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity, association, risk factor') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity, association') \
                              | (clinvar['ClinicalSignificance'] =='conflicting data from submitters') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity, drug response, other') \
                              | (clinvar['ClinicalSignificance'] =='Uncertain significance, drug response') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity, drug response') \
                              | (clinvar['ClinicalSignificance'] =='Uncertain significance, Affects') \
                              | (clinvar['ClinicalSignificance'] =='association not found' ) \
                              | (clinvar['ClinicalSignificance'] =='Uncertain significance, other') \
                              | (clinvar['ClinicalSignificance'] =='Uncertain significance, association') \
                              | (clinvar['ClinicalSignificance'] =='Conflicting interpretations of pathogenicity, risk factor' ) \
                              | (clinvar['ClinicalSignificance'] =='no interpretation for the single variant' )]

In [ ]:
clinvar_missense_nonsense_silent= clinvar.loc[(clinvar['Type'] == 'Variation') \
                                | (clinvar['Type'] == 'single nucleotide variant') \
                                | (clinvar['Type'] == 'protein only')]

# keep only germline, one AA subtitution and confirmed
germline_missense_nonsense_silent = clinvar_missense_nonsense_silent[clinvar_missense_nonsense_silent['OriginSimple']=='germline']
germline_confirm = germline_missense_nonsense_silent.loc[(germline_missense_nonsense_silent['ReviewStatus'] == 'criteria provided, multiple submitters, no conflicts') \
                              | (germline_missense_nonsense_silent['ReviewStatus'] == 'criteria provided, single submitter') \
                              | (germline_missense_nonsense_silent['ReviewStatus'] == 'practice guideline') \
                              | (germline_missense_nonsense_silent['ReviewStatus'] == 'reviewed by expert panel') ]
germline_confirm = germline_confirm[germline_confirm['Name'].str.contains(r'\+') == False]
germline_confirm = germline_confirm[germline_confirm['Name'].str.contains('-') == False]
germline_confirm = germline_confirm[germline_confirm['Name'].str.contains('g') == False] # upstream variants
germline_confirm = germline_confirm[germline_confirm['Name'].str.contains('p.') == True]
germline_confirm_missense = germline_confirm[germline_confirm['Name'].str.contains('=') == False]
germline_confirm_missense = germline_confirm_missense[germline_confirm_missense['Name'].str.contains('\+') == False]
germline_confirm_missense = germline_confirm_missense[germline_confirm_missense['Name'].str.contains('\*') == False]

In [ ]:
benign_deleterious= germline_confirm_missense.loc[(germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Benign/Likely benign') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Benign') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Likely benign') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Benign, other') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic, risk factor') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Likely pathogenic, other') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic, risk factor') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic, drug response') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Pathogenic, drug response') \
                              | (germline_confirm_missense['ClinicalSignificance'] == 'Benign/Likely benign, other') ]

# Rename categories Likely Pathogenic etc to either Pathogenic (1) or Benign (0)
benign_deleterious['ClinicalSignificance'] = benign_deleterious['ClinicalSignificance'].replace(['Pathogenic','Pathogenic/Likely pathogenic', 'Pathogenic, risk factor', 'Likely pathogenic, other', 'Pathogenic/Likely pathogenic, drug response', 'Pathogenic, drug response' ], 1)
benign_deleterious['ClinicalSignificance'] = benign_deleterious['ClinicalSignificance'].replace(['Benign/Likely benign', 'Benign', 'Likely benign', 'Benign/Likely benign, other', 'Benign, other' ], 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
len(benign_deleterious)

61418

In [ ]:
benign_deleterious = benign_deleterious.reset_index(drop=True)

In [ ]:
benign_deleterious.head()

,Type,Name,GeneSymbol,ClinicalSignificance,OriginSimple,ReviewStatus
0,single nucleotide variant,NM_017547.4(FOXRED1):c.694C>T (p.Gln232Ter),FOXRED1,1,germline,"criteria provided, multiple submitters, no con..."
1,single nucleotide variant,NM_000410.4(HFE):c.848A>C (p.Gln283Pro),HFE,1,germline,"criteria provided, single submitter"
2,single nucleotide variant,NM_138413.4(HOGA1):c.860G>T (p.Gly287Val),HOGA1,1,germline,"criteria provided, multiple submitters, no con..."
3,single nucleotide variant,NM_138413.4(HOGA1):c.769T>G (p.Cys257Gly),HOGA1,1,germline,"criteria provided, multiple submitters, no con..."
4,single nucleotide variant,NM_001194998.2(CEP152):c.794A>C (p.Gln265Pro),CEP152,1,germline,"criteria provided, multiple submitters, no con..."


In [ ]:
list_AA_orig, list_AA_mut, list_position, list_refseq = [], [], [], []

for ind, row in benign_deleterious.iterrows():
    temp = row['Name']
    temp = temp.split('(')
    string = temp[2]
    name = temp[0]
    if '=' in string:
      string = string[2:-1]
      position = string[3:-1]
      aa_orig = string[:3]
      aa_mut = aa_orig
    else:
      string = string[2:-1]
      position = string[3:-3]
      aa_orig = string[:3]
      aa_mut = string[-3:]
    list_AA_orig.append(AAconvert(aa_orig))
    list_AA_mut.append(AAconvert(aa_mut))
    list_position.append(position)
    list_refseq.append(name)

In [ ]:
benign_deleterious['AA_orig'] = list_AA_orig
benign_deleterious['AA_targ'] = list_AA_mut
benign_deleterious['position'] = list_position
benign_deleterious['refseq'] = list_refseq

In [ ]:
benign_deleterious_missense = benign_deleterious[benign_deleterious['AA_targ'] != '*']

In [ ]:
mart = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/mart_export_refseq_uniprot_isoform.txt", '\t')
mart =  mart.drop_duplicates(subset=['RefSeq mRNA ID','UniProtKB isoform ID','UniProtKB/Swiss-Prot ID' ], keep='first')
mart = mart.dropna(subset=['UniProtKB isoform ID', 'UniProtKB/Swiss-Prot ID'], how = 'all')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
mart_temp

,Transcript stable ID version,RefSeq mRNA ID,UniProtKB isoform ID,UniProtKB/Swiss-Prot ID,Gene stable ID version
6463,ENST00000383118.8,NM_001290043,Q03519-1,Q03519,ENSG00000206235.8
10889,ENST00000419142.6,NM_001290043,NaN,Q03519,ENSG00000237599.9


In [ ]:
## Find uniprot id (isoform)

# 1) search on mart_export RefSeq mRNA ID --> UniProtKB isoform ID, if not found:
# 2) gene -- > uniprot --> download all transcripts from uniprot website --> test if AA_orig at the right place
benign_deleterious_missense = benign_deleterious_missense.reset_index(drop=True)
transcript_not_found, delins, unknown_genes, count_test = 0, 0, 0, 0
sequence, indices_to_excl, isoform_uniprot = [], [], []
count_found, count_not_found = 0, 0
count_t = 0
for i, mut in benign_deleterious_missense.iterrows():
  count_t+=1
  if count_t>200:
    break

  refseq = mut['refseq'].split('.')
  refseq = refseq[0]
  mart_temp = mart[mart['RefSeq mRNA ID'] ==refseq]
  AA_orig = mut['AA_orig']
  AA_targ = mut['AA_targ']
  pos_temp = int(mut['position'])

  # 1)
  if len(mart_temp)>0: # refseq symbol found in mart export
    if len(mart_temp) >1:
      mart_temp = mart_temp.dropna(subset=['UniProtKB isoform ID', 'UniProtKB/Swiss-Prot ID'], how = 'any')

    if notNaN(mart_temp['UniProtKB isoform ID']).values:
      uniprot_id = mart_temp['UniProtKB isoform ID'].item()
   
      transcript = load_uniprot_fasta(uniprot_id)
 
      if pos_temp <= len(transcript):

        if transcript[pos_temp - 1] == AA_orig:
          isoform_name = uniprot_id
          mut_seq = changeAAnew(uniprot_id,AA_orig, AA_targ, pos_temp, transcript)
          isoform_uniprot.append(isoform_name)
          #position.append(pos_temp)
          #AA_orig_list.append(AA_orig)
          #AA_targ_list.append(AA_targ)
          count_found +=1
          sequence.append(mut_seq) 

        else:
          transcript_not_found +=1
          indices_to_excl.append(i)

      else:
          indices_to_excl.append(i)
 
 
    elif notNaN(mart_temp['UniProtKB/Swiss-Prot ID']).values:
      uniprot_canonical = mart_temp['UniProtKB/Swiss-Prot ID'].item()
      # load all transcript from uniprot
      all_transcripts = load_uniprot_fasta_all(uniprot_canonical)
      count = 1 # track transcripts

      for isoform, transcript in enumerate(all_transcripts):

        if pos_temp <= len(transcript):

            if transcript[pos_temp - 1] == AA_orig:
              isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
              mut_seq = changeAAnew(uniprot_canonical, AA_orig, AA_targ, pos_temp, transcript)
              isoform_uniprot.append(isoform_name)
              #position.append(pos_temp)
              #AA_orig_list.append(AA_orig)
              #AA_targ_list.append(AA_targ)
              count_found +=1
              sequence.append(mut_seq) 
              break

            else:
              if len(all_transcripts) == count:
                transcript_not_found +=1
                indices_to_excl.append(i)
              count+=1

        else:
            if len(all_transcripts) == count:
              transcript_not_found +=1
              indices_to_excl.append(i)
            count+=1


  # 2)
  else:

    gene_name = mut['GeneSymbol']

    if gene_name in gene_dict:
      uniprot_canonical_list = gene_dict[gene_name] # we may have more than one corresponding proteins for gene
    
    elif gene_name in gene_synonym_dict:
      uniprot_canonical_list = gene_synonym_dict[gene_name] 

    else:
      unknown_genes +=1
      indices_to_excl.append(i)
      continue
    
    count_uniprot =0 # track isoforms
    found=False # used to stop searching follwing isoforms

    for uniprot_canonical in uniprot_canonical_list:
      count_uniprot+=1 
      if found == True:
        break

      # load all transcript from uniprot
      all_transcripts = load_uniprot_fasta_all(uniprot_canonical)
      count = 1 # track transcripts
      for isoform, transcript in enumerate(all_transcripts):

        if pos_temp <= len(transcript):

            if transcript[pos_temp - 1] == string[2]:
              isoform_name = uniprot_canonical+ '-' + str(isoform+1) 
              mut_seq = changeAAnew(gene_name, AA_orig, AA_targ, pos_temp, transcript)
              isoform_uniprot.append(isoform_name)
              #position.append(pos_temp)
              #AA_orig_list.append(orig)
              #AA_targ_list.append(AA_targ)
              count_found +=1
              found = True
              sequence.append(mut_seq) 
              break

            else:
              if len(all_transcripts) == count and count_uniprot == len(uniprot_canonical_list):
                transcript_not_found +=1
                indices_to_excl.append(i)
              count+=1

        else:
            if len(all_transcripts) == count and count_uniprot == len(uniprot_canonical_list):
              transcript_not_found +=1
              indices_to_excl.append(i)
            count+=1

In [ ]:
print(len(isoform_uniprot), len(sequence))

196 196


In [ ]:
len(indices_to_excl)

4

In [ ]:
len(benign_deleterious_missense)

In [ ]:
benign_deleterious_missense = benign_deleterious_missense.drop(benign_deleterious_missense.index[indices_to_excl])
benign_deleterious_missense['mut_sequence'] = sequence # Mutated sequence
benign_deleterious_missense['uniprot'] = isoform_uniprot

0

In [ ]:
# save mutations
benign_deleterious_missense.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/clinvar_germline_missense.csv', index=False)

In [ ]:
# Are there mutations on both Benign and Pathogenic category? No
# Exclude them and put them to conflicting

conflict = germline_confirm_missense.loc[(germline_confirm_missense['ClinicalSignificance'] == 'Uncertain significance, other') \
                                    | (germline_confirm_missense['ClinicalSignificance'] == 'Conflicting interpretations of pathogenicity, drug response') \
                                    | (germline_confirm_missense['ClinicalSignificance'] == 'Conflicting interpretations of pathogenicity') \
                                    | (germline_confirm_missense['ClinicalSignificance'] ==  'Uncertain significance, risk factor') ]

print(len(conflict))

4


## DRGN

In [ ]:
DRGN = pd.read_csv("/content/drive/MyDrive/my_colab/3rdYear/datasets/DRGN.csv", sep = '\t', names = ['uniprot', 'mutation', 'label'])

In [ ]:
wt_sequences, mut_sequences, uniprot_list, AA_orig_list, AA_targ_list, position, label = [], [], [], [], [], [], []
test, test2=0,0
for i, mut in DRGN.iterrows():
      AA_orig = mut['mutation'][0]
      AA_targ = mut['mutation'][-1]
      pos_temp = int(mut['mutation'][1:-1])
          
      # load all transcript from uniprot
      all_transcripts, iso_names = load_uniprot_fasta_all(mut['uniprot'])
      
      count = 1 # track transcripts
      for isoform, transcript in enumerate(all_transcripts):

        if pos_temp <= len(transcript):

            if transcript[pos_temp - 1] == AA_orig:
              #isoform_name = mut['uniprot']+ '-' + str(isoform+1) 
              if isoform == 0:
                isoform_name = iso_names[isoform] + '-1'
              else:
                isoform_name = iso_names[isoform]
              mut_seq = changeAAnew(mut['uniprot'], AA_orig, AA_targ, pos_temp, transcript)
              uniprot_list.append(isoform_name)
              position.append(pos_temp)
              AA_orig_list.append(AA_orig)
              AA_targ_list.append(AA_targ)
              mut_sequences.append(mut_seq) 
              wt_sequences.append(transcript)
              label.append(mut['label'])
              break

            else:
              if len(all_transcripts) == count:
                test+=1
              count+=1

        else:
            #print('position bigger than length')
            if len(all_transcripts) == count:
              test2+=1
            count+=1

In [ ]:
len(mut_sequences)

3644

In [ ]:
drgn_mutations = pd.DataFrame(list(zip(uniprot_list, wt_sequences, mut_sequences, AA_orig_list, position, AA_targ_list, label)),
               columns =['uniprot id', 'WT_sequence', 'mut_sequence', 'AA_orig', 'position', 'AA_targ', 'label'])

drgn_mutations.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/DRGN_mutations.csv', index=False)

## CPV

In [ ]:
CPV = pd.read_csv("/content/drive/MyDrive/my_colab/*2ndYear/CPVlist.csv", sep = ',', usecols = ['Protein', 'Mutation', 'Label'])

In [ ]:
CPV.head()

,Protein,Mutation,Label
0,Q9UM73,F1174L,1
1,Q9UM73,I1461V,0
2,Q9UM73,K1491R,0
3,Q9UM73,R1275Q,1
4,P15056,G469A,1


In [ ]:
wt_sequences, mut_sequences, uniprot_list, AA_orig_list, AA_targ_list, position, label = [], [], [], [], [], [], []
test, test2=0,0
for i, mut in CPV.iterrows():
      AA_orig = mut['Mutation'][0]
      AA_targ = mut['Mutation'][-1]
      pos_temp = int(mut['Mutation'][1:-1])
          
      # load all transcript from uniprot
      all_transcripts, iso_names = load_uniprot_fasta_all(mut['Protein'])
      
      count = 1 # track transcripts
      for isoform, transcript in enumerate(all_transcripts):

        if pos_temp <= len(transcript):

            if transcript[pos_temp - 1] == AA_orig:
              #isoform_name = mut['uniprot']+ '-' + str(isoform+1) 
              if isoform == 0:
                isoform_name = iso_names[isoform] + '-1'
              else:
                isoform_name = iso_names[isoform]
              mut_seq = changeAAnew(mut['Protein'], AA_orig, AA_targ, pos_temp, transcript)
              uniprot_list.append(isoform_name)
              position.append(pos_temp)
              AA_orig_list.append(AA_orig)
              AA_targ_list.append(AA_targ)
              mut_sequences.append(mut_seq) 
              wt_sequences.append(transcript)
              label.append(mut['Label'])
              break

            else:
              if len(all_transcripts) == count:
                test+=1
              count+=1

        else:
            #print('position bigger than length')
            if len(all_transcripts) == count:
              test2+=1
            count+=1

In [ ]:
len(CPV)

244

In [ ]:
cpv_mutations = pd.DataFrame(list(zip(uniprot_list, wt_sequences, mut_sequences, AA_orig_list, position, AA_targ_list, label)),
               columns =['uniprot id', 'WT_sequence', 'mut_sequence', 'AA_orig', 'position', 'AA_targ', 'label'])

cpv_mutations.to_csv('/content/drive/MyDrive/my_colab/3rdYear/datasets/CPV_mutations.csv', index=False)